# generate assets



```markdown
# BigQuery & Gemini: Generating and Analyzing Multimodal Data

This notebook demonstrates how to use Vertex AI's generative models to create a multimodal dataset, store it in Google Cloud Storage, and then analyze it using BigQuery and Gemini.
```


```markdown
## 1. Setup and Installation

First, let's install the necessary Python libraries for interacting with Google Cloud services.
```

In [1]:
%pip install --upgrade --user google-cloud-aiplatform google-cloud-storage google-cloud-bigquery


```markdown
Next, please fill in your Google Cloud project details and other configuration values below.
```

In [ ]:
import vertexai
from vertexai.preview.vision_models import ImageGenerationModel
from google.cloud import texttospeech, storage, bigquery
import json
import os

In [3]:
#easy test - to be deleted later
import os

# Your Google Cloud project ID
PROJECT_ID = "geminienterprise-485114"
# The region for your resources
LOCATION = "us-central1"
# Your Google Cloud Storage bucket name
GCS_BUCKET = " meetupmarch"
# Your BigQuery dataset name
# BIGQUERY_DATASET = "your_bigquery_dataset"

# Authenticate with Google Cloud
if "google.colab" in str(get_ipython()):
    from google.colab import auth
    auth.authenticate_user()

# Initialize Vertex AI
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# Authentication
# --- AUTHENTICATE ---
# Authenticate with Google Cloud. This is crucial for running in a Colab environment.
# It will trigger a pop-up window to ask for your credentials and permissions.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
# --- CONFIGURATION FOR YOUR MEETUP ---
# Define project-wide variables.
import os
import vertexai
from google.cloud import texttospeech, storage, bigquery
import json

# Your Google Cloud project ID
PROJECT_ID = "geminienterprise-485114"

# The region for your resources
LOCATION = "us-central1"

# Your Google Cloud Storage bucket name (no 'gs://' prefix)
GCS_BUCKET_NAME = "meetupmarch"

# Derived names for our BigQuery resources
DATASET_ID = "generative_assets_dataset"
TABLE_ID = "assets_metadata"

# --- INITIALIZE CLIENTS ---
# Initialize Vertex AI SDK and other clients with your project details.
# After authentication, these clients will have the necessary permissions.
vertexai.init(project=PROJECT_ID, location=LOCATION)

storage_client = storage.Client(project=PROJECT_ID)
bq_client = bigquery.Client(project=PROJECT_ID)
tts_client = texttospeech.TextToSpeechClient()

print(f"Project: {PROJECT_ID}, Location: {LOCATION}")
print("Vertex AI and other Google Cloud clients initialized successfully.")



In [ ]:
# --- PREPARE GCS BUCKET AND BIGQUERY DATASET ---
# This code will create the resources if they don't already exist.

# GCS Bucket
bucket = storage_client.bucket(GCS_BUCKET_NAME)
if not bucket.exists():
    bucket.create(location=LOCATION)
    print(f"Bucket '{GCS_BUCKET_NAME}' created.")
else:
    print(f"Bucket '{GCS_BUCKET_NAME}' already exists.")

In [ ]:
# BigQuery Dataset
dataset_ref = bq_client.dataset(DATASET_ID)
try:
    bq_client.get_dataset(dataset_ref)
    print(f"Dataset '{DATASET_ID}' already exists.")
except Exception:
    bq_client.create_dataset(dataset_ref)
    print(f"Dataset '{DATASET_ID}' created.")


In [ ]:
# A list to hold metadata for all generated assets
all_metadata = []

```markdown
## 2. Data Generation with Vertex AI

Now, let's generate some multimodal data using different Vertex AI models.
```

```markdown
### 2.1 Generate an Image with Imagen - 1 image
```

In [7]:
import vertexai
from vertexai.vision_models import ImageGenerationModel

# TODO: Specify your project ID and location
# vertexai.init(project="your-project-id", location="your-location")

image_model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-002")

image_prompt = "a futuristic banana-shaped spaceship flying through a nebula"

response = image_model.generate_images(prompt=image_prompt)

# The response is a list of Image objects.
# Access the first image directly and save it.
response[0].save("generated_image.png")

print("Image generated and saved as generated_image.png")

Image generated and saved as generated_image.png


```markdown
### 2.1 Generate an Image with Imagen - 10 images
```

In [ ]:
print("--- Starting Image Generation (Imagen 3) ---")
image_model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-002")
local_image_dir = "generated_images"
os.makedirs(local_image_dir, exist_ok=True)

image_prompts = [
    "A state-of-the-art chemical manufacturing plant at sunset, with clean energy sources visible.",
    "Macro shot of a new, sustainable consumer goods product made from plant-based materials.",
    "A team of engineers in a modern factory reviewing data on a holographic display.",
    "Futuristic robotic arms assembling a complex piece of machinery with precision.",
    "A digital twin of a manufacturing facility, showing real-time operational data streams.",
    "An aerial view of a smart warehouse with autonomous forklifts and delivery drones.",
    "A scientist in a lab coat examining a beaker with a glowing liquid.",
    "High-end cosmetic products arranged in a minimalist, elegant composition.",
    "A cross-section of an advanced engine, showing intricate inner workings.",
    "A beautiful landscape shot of a factory that blends seamlessly with nature.",
]

for i, prompt in enumerate(image_prompts):
    local_filename = f"{local_image_dir}/image_{i}.png"
    gcs_blob_name = f"images/image_{i}.png"

    print(f"Generating image {i+1}/10 with prompt: '{prompt[:50]}...'")
    response = image_model.generate_images(prompt=prompt)
    response[0].save(local_filename)

    # Upload to GCS
    blob = bucket.blob(gcs_blob_name)
    blob.upload_from_filename(local_filename)
    gcs_uri = f"gs://{GCS_BUCKET_NAME}/{gcs_blob_name}"

    # Store metadata
    all_metadata.append({
        "asset_id": f"image_{i}",
        "asset_type": "image",
        "prompt": prompt,
        "gcs_uri": gcs_uri,
        "model_used": "imagen-3.0-generate-002"
    })
    print(f"Image {i+1} saved and uploaded to {gcs_uri}")

print("--- Image Generation Complete ---")


```markdown
### 2.2 Generate Music with Lyria
```

In [ ]:
print("\n--- Starting Music Generation (Lyria) ---")
from vertexai.preview.generative_models import GenerativeModel, MusicGenerationInput

# --- CONFIGURATION for Lyria ---
# As per the official Colab notebook, "music-generation-preview" is the correct model endpoint.
lyria_model = GenerativeModel("music-generation-preview")
local_music_dir = "generated_music"
os.makedirs(local_music_dir, exist_ok=True)

# Prompts designed to evoke a corporate, industrial, or tech-focused mood
music_prompts = [
    "An optimistic and inspiring corporate anthem, with a steady electronic beat and piano.",
    "A minimal, ambient electronic track for a technology product showcase, calm and focused.",
    "A powerful, driving industrial beat with synth elements, suggesting innovation and power.",
    "An atmospheric and thoughtful soundscape for a documentary about sustainable manufacturing.",
    "A futuristic, high-energy electronic track with a sense of speed and progress, for a product launch video.",
    "A gentle, flowing piano and strings piece, evoking precision and care, for a chemical goods company ad.",
    "An 8-bit chiptune track with a modern synth bassline, representing the gamification of enterprise software.",
    "A tense, percussive track for a cybersecurity-themed video, creating a sense of urgency.",
    "A warm, uplifting acoustic guitar track for an internal company video about teamwork and success.",
    "A cinematic, orchestral piece with a grand and epic feel, for a brand story video.",
]

for i, prompt in enumerate(music_prompts):
    local_filename = f"{local_music_dir}/music_{i}.wav"
    gcs_blob_name = f"music/music_{i}.wav"
    clip_duration = 20  # Set a consistent duration for each clip

    print(f"Generating music {i+1}/10 with prompt: '{prompt[:50]}...'")

    # Use the structured MusicGenerationInput as shown in the Colab notebook
    music_input = MusicGenerationInput(prompt=prompt, duration_secs=clip_duration)

    # The generate_content method expects a list of inputs
    response = lyria_model.generate_content([music_input])

    # Safely extract the audio data by checking the MIME type, as recommended
    audio_data = None
    for part in response.candidates[0].content.parts:
        if part.mime_type == "audio/wav":
            audio_data = part.data
            break

    if not audio_data:
        print(f"Warning: Could not extract audio data for prompt {i+1}. Skipping.")
        continue

    # Save the raw audio bytes to a file
    with open(local_filename, "wb") as f:
        f.write(audio_data)

    # Upload to GCS
    blob = bucket.blob(gcs_blob_name)
    blob.upload_from_filename(local_filename)
    gcs_uri = f"gs://{GCS_BUCKET_NAME}/{gcs_blob_name}"

    # Store metadata with the correct model name
    all_metadata.append({
        "asset_id": f"music_{i}",
        "asset_type": "music",
        "prompt": prompt,
        "gcs_uri": gcs_uri,
        "model_used": "music-generation-preview" # Updated model name
    })
    print(f"Music {i+1} saved and uploaded to {gcs_uri}")

print("--- Music Generation Complete ---")


```markdown
### 2.3 Generate Speech with Gemini TTS
```

In [ ]:
# generate 1 file only test
from google.cloud import texttospeech

client = texttospeech.TextToSpeechClient()

synthesis_input = texttospeech.SynthesisInput(text="Hello, this is a test of the Gemini Text-to-Speech API.")
voice = texttospeech.VoiceSelectionParams(language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL)
audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)

response = client.synthesize_speech(input=synthesis_input, voice=voice, audio_config=audio_config)

with open("generated_speech.mp3", "wb") as out:
    out.write(response.audio_content)
    print('Audio content written to file "generated_speech.mp3"')

In [ ]:
# generate 10 files
print("\n--- Starting Speech Generation (Text-to-Speech) ---")
local_speech_dir = "generated_speech"
os.makedirs(local_speech_dir, exist_ok=True)

speech_texts = [
    "Quarterly production targets have been exceeded by fifteen percent.",
    "Safety protocol update: All personnel must attend the mandatory briefing on Friday.",
    "The new supply chain optimization model is now live across all regions.",
    "Alert: Unscheduled maintenance is required for assembly line three.",
    "Our commitment to sustainable manufacturing has reduced our carbon footprint by 20% year-over-year.",
    "The next shareholder meeting will be held on July 25th to discuss Q2 earnings.",
    "Innovation in materials science is key to developing our next generation of products.",
    "Customer feedback indicates a 95% satisfaction rate with our new service portal.",
    "We are projecting a 10% growth in the consumer goods sector for the upcoming fiscal year.",
    "Emergency shutdown procedures for the chemical processing unit have been initiated. This is a drill.",
]

for i, text in enumerate(speech_texts):
    local_filename = f"{local_speech_dir}/speech_{i}.mp3"
    gcs_blob_name = f"speech/speech_{i}.mp3"

    print(f"Generating speech {i+1}/10 for text: '{text[:50]}...'")
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL)
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)

    response = tts_client.synthesize_speech(input=synthesis_input, voice=voice, audio_config=audio_config)

    with open(local_filename, "wb") as out:
        out.write(response.audio_content)

    # Upload to GCS
    blob = bucket.blob(gcs_blob_name)
    blob.upload_from_filename(local_filename)
    gcs_uri = f"gs://{GCS_BUCKET_NAME}/{gcs_blob_name}"

    # Store metadata
    all_metadata.append({
        "asset_id": f"speech_{i}",
        "asset_type": "speech",
        "prompt": text,
        "gcs_uri": gcs_uri,
        "model_used": "google-text-to-speech"
    })
    print(f"Speech {i+1} saved and uploaded to {gcs_uri}")

print("--- Speech Generation Complete ---")


```markdown
## 3. Upload to Google Cloud Storage
```

In [ ]:
# easy version
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.bucket(GCS_BUCKET)

def upload_to_gcs(filename):
    blob = bucket.blob(filename)
    blob.upload_from_filename(filename)
    return f"gs://{GCS_BUCKET}/{filename}"

image_gcs_uri = upload_to_gcs("generated_image.png")
music_gcs_uri = upload_to_gcs("generated_music_prompt.txt")
speech_gcs_uri = upload_to_gcs("generated_speech.mp3")

print(f"Image URI: {image_gcs_uri}")
print(f"Music URI: {music_gcs_uri}")
print(f"Speech URI: {speech_gcs_uri}")

#  BigQuery Metadata Ingestion

In [ ]:
print("\n--- Creating and Uploading Metadata File ---")
local_metadata_filename = "metadata.jsonl"
gcs_metadata_blob_name = "metadata/assets.jsonl"

with open(local_metadata_filename, "w") as f:
    for item in all_metadata:
        f.write(json.dumps(item) + "\n")

blob = bucket.blob(gcs_metadata_blob_name)
blob.upload_from_filename(local_metadata_filename)
metadata_gcs_uri = f"gs://{GCS_BUCKET_NAME}/{gcs_metadata_blob_name}"

print(f"Metadata file uploaded to {metadata_gcs_uri}")


# Create BigQuery External Table

In [ ]:
print("\n--- Creating BigQuery External Table ---")

table_ref = dataset_ref.table(TABLE_ID)

# Define the schema for the external table
schema = [
    bigquery.SchemaField("asset_id", "STRING"),
    bigquery.SchemaField("asset_type", "STRING"),
    bigquery.SchemaField("prompt", "STRING"),
    bigquery.SchemaField("gcs_uri", "STRING"),
    bigquery.SchemaField("model_used", "STRING"),
]

external_config = bigquery.ExternalConfig("JSON")
external_config.source_uris = [metadata_gcs_uri]
external_config.schema = schema
# For JSONL, autodetect often works well, but explicit schema is safer.
# external_config.autodetect = True

# Create the table
try:
    bq_client.delete_table(table_ref, not_found_ok=True) # Delete if it exists to ensure a fresh start
    print(f"Existing table '{TABLE_ID}' deleted.")
except Exception as e:
    print(e)

table = bigquery.Table(table_ref)
table.external_data_configuration = external_config
table = bq_client.create_table(table)

print(f"External table '{table.project}.{table.dataset_id}.{table.table_id}' created successfully.")


# Analyze Data with BigQuery and Gemini

Before running the SQL: You'll need a reference to a Gemini model in BigQuery. If you don't have one, you can create it with this DDL command in BigQuery:

In [ ]:
print("\n--- Creating BigQuery Remote Model ---")

# --- CONFIGURATION for BigQuery Connection ---
# !!! IMPORTANT !!!
# Please replace this with the actual ID of your BigQuery connection.
# The connection must be created in the same location as your dataset (e.g., 'us-central1').
CONNECTION_NAME = "your-bq-connection-to-vertex-ai" # e.g., "bq-vertex-connection"

# Define the SQL query to create the remote model
sql_create_model = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.gemini_pro_vision_model`
REMOTE WITH CONNECTION `{PROJECT_ID}.{LOCATION}.{CONNECTION_NAME}`
OPTIONS (remote_service_type = 'CLOUD_AI_LARGE_LANGUAGE_MODEL_V1');
"""

print("Executing query to create or replace BigQuery remote model...")
# Execute the query using the BigQuery client
query_job = bq_client.query(sql_create_model)
query_job.result()  # Wait for the job to complete

print(f"BigQuery remote model `{DATASET_ID}.gemini_pro_vision_model` created or replaced successfully.")



In [ ]:
print("\n--- Analyzing Images with Gemini in BigQuery ---")

# The fully qualified ID of the table we created earlier
full_table_id = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

# This query selects only the image assets, then passes their GCS URI and
# the original prompt to the Gemini model for analysis.
sql_analyze = f"""
SELECT
    prompt,
    gcs_uri,
    ml_generate_text_result['predictions'][0]['content'] AS gemini_analysis
FROM
    ML.GENERATE_TEXT(
        MODEL `{PROJECT_ID}.{DATASET_ID}.gemini_pro_vision_model`,
        (
            -- Subquery to select only the images and their URIs
            SELECT
                prompt,
                gcs_uri
            FROM
                `{full_table_id}`
            WHERE
                asset_type = 'image'
        ),
        STRUCT(
            -- This is the prompt for the LLM analysis itself
            'Describe this image in detail based on its content. Also, comment on how well it matches the original user prompt.' AS prompt,
            TRUE AS flatten_json_output
        )
    );
"""

print("Executing analysis query... This may take a few moments.")
# Execute the query and load the results into a pandas DataFrame
df = bq_client.query(sql_analyze).to_dataframe()

# Display the results in a clean markdown format
print("\n--- Gemini Vision Analysis Results ---")
print(df.to_markdown(index=False))



```markdown
## 4. Create BigQuery Table

Now we'll create a JSONL file with metadata about our generated assets and upload it to GCS. Then we'll create a BigQuery external table that points to this metadata file.
```

In [ ]:
import json

metadata = [
    {"prompt": image_prompt, "gcs_uri": image_gcs_uri, "type": "image"},
    {"prompt": music_prompt, "gcs_uri": music_gcs_uri, "type": "music"},
    {"prompt": "Hello, this is a test of the Gemini Text-to-Speech API.", "gcs_uri": speech_gcs_uri, "type": "speech"}
]

with open("metadata.jsonl", "w") as f:
    for item in metadata:
        f.write(json.dumps(item) + "\n")

metadata_gcs_uri = upload_to_gcs("metadata.jsonl")
print(f"Metadata GCS URI: {metadata_gcs_uri}")

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client()

dataset_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}"
try:
    bq_client.get_dataset(dataset_id)  # Make an API request.
    print(f"Dataset {dataset_id} already exists.")
except Exception:
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = LOCATION
    dataset = bq_client.create_dataset(dataset, timeout=30)
    print(f"Created dataset {PROJECT_ID}.{dataset.dataset_id}")

table_id = f"{dataset_id}.multimodal_assets"
schema = [
    bigquery.SchemaField("prompt", "STRING"),
    bigquery.SchemaField("gcs_uri", "STRING"),
    bigquery.SchemaField("type", "STRING"),
]

external_config = bigquery.ExternalConfig("NEWLINE_DELIMITED_JSON")
external_config.source_uris = [metadata_gcs_uri]
external_config.schema = schema
table = bigquery.Table(table_id, schema=schema)
table.external_data_configuration = external_config
table = bq_client.create_table(table, exists_ok=True)

print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")

```markdown
## 5. Analyze Data with BigQuery and Gemini

Finally, we'll create a remote model in BigQuery that points to the Gemini Pro Vision model. This will allow us to analyze the images directly from BigQuery using SQL.
```

In [ ]:
# This step assumes you have a BigQuery connection to Vertex AI set up.
# See: https://cloud.google.com/bigquery/docs/create-cloud-resource-connection
CONNECTION_NAME = "your-bq-connection-to-vertex-ai"

sql_create_model = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{BIGQUERY_DATASET}.gemini_vision_model`
REMOTE WITH CONNECTION `{PROJECT_ID}.{LOCATION}.{CONNECTION_NAME}`
OPTIONS (remote_service_type = 'CLOUD_AI_LARGE_LANGUAGE_MODEL_V1');
"""

query_job = bq_client.query(sql_create_model)
query_job.result()  # Wait for the job to complete
print("BigQuery remote model created.")

In [ ]:
sql_analyze = f"""
SELECT
    prompt,
    gcs_uri,
    ml_generate_text_result['predictions'][0]['content'] AS gemini_analysis
FROM
    ML.GENERATE_TEXT(
        MODEL `{PROJECT_ID}.{BIGQUERY_DATASET}.gemini_vision_model`,
        (SELECT prompt, gcs_uri FROM `{table_id}` WHERE type = 'image'),
        STRUCT('Analyze the following image:' AS prompt, TRUE AS flatten_json_output)
    );
"""

df = bq_client.query(sql_analyze).to_dataframe()
print(df.to_markdown())